In [1]:
import random

pesos = [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120]
ganancias = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
CapacidadMochila = 750
NumGen = 100 # numero de generaciones
TamHormigas = 500 # Tamano de la colonia
TasaEvaporacion = 0.5

# --- Corregido ---
# Nivel de feromona inicial (float para precision)
Feromonas = [1.0] * len(pesos)
# Trackea la mejor ganancia encontrada
MejorSolucion = 0
# Trackea QUÉ objetos componen la mejor solución
MejorSolucionVector = [0] * len(pesos)

In [ ]:
**Algoritmo principal

In [3]:
print("Generaciones: ")

# Lista para almacenar las ganancias de todas las soluciones encontradas en cada generacion
all_gains = []

for i in range(NumGen):
    print(f"{i} / {NumGen}", end="\r")

    # Almacenará los resultados de todas las hormigas en ESTA generacion
    # Guardaremos tuplas: (ganancia_total, vector_solucion)
    ColHormigas = []

    for j in range(TamHormigas):
        # --- Cada hormiga construye su propia solucion ---
        ant_solution = [0] * len(pesos) # Mochila de esta hormiga (0=no, 1=si)
        ant_weight = 0
        ant_gain = 0

        # Suma total de feromonas para calcular probabilidades
        # (Se añade 1e-10 para evitar division por cero si todo se evapora)
        total_feromonas = sum(Feromonas) + 1e-10

        # Crear una lista de indices de objetos y barajarla para introducir aleatoriedad en el orden de seleccion
        item_indices = list(range(len(pesos)))
        random.shuffle(item_indices)

        for k in item_indices: # Iterar sobre los indices barajados
            # Probabilidad de elegir el objeto k
            probabilidad = Feromonas[k] / total_feromonas

            r = random.random()

            # La hormiga decide tomar el objeto
            if r < probabilidad:
                # PERO, solo lo toma si cabe en la mochila
                if ant_weight + pesos[k] <= CapacidadMochila:
                    ant_solution[k] = 1 # Marcar como "tomado"
                    ant_weight += pesos[k]
                    ant_gain += ganancias[k]

        # La hormiga terminó. Guardamos su resultado (ganancia y mochila)
        ColHormigas.append( (ant_gain, ant_solution) )
        # Agregar la ganancia de esta hormiga a la lista de todas las ganancias
        all_gains.append(ant_gain)


    # --- Fin del bucle de hormigas (j) ---
    # Todas las hormigas de esta generacion han terminado

    # Encontrar la mejor hormiga DE ESTA GENERACION
    best_gen_ant = max(ColHormigas, key=lambda x: x[0])
    best_gen_gain = best_gen_ant[0]
    best_gen_vector = best_gen_ant[1]

    # Actualizar la MEJOR SOLUCION GLOBAL si la de esta generacion es mejor
    if best_gen_gain > MejorSolucion:
        MejorSolucion = best_gen_gain
        MejorSolucionVector = best_gen_vector.copy() # Importante copiar la lista

    # --- Actualizacion de Feromonas ---

    # 1. Evaporacion (para todos los objetos)
    for k in range(len(Feromonas)):
        Feromonas[k] *= (1 - TasaEvaporacion)

    # 2. Deposito (solo en los objetos de la MEJOR solucion GLOBAL encontrada hasta ahora)
    # Depositamos una cantidad proporcional a la ganancia de la MEJOR solucion GLOBAL
    for k in range(len(pesos)):
        if MejorSolucionVector[k] == 1:
            Feromonas[k] += MejorSolucion # Reforzar caminos exitosos


    # Condicion de parada (si encontramos la solucion optima conocida para este set)
    if MejorSolucion >= 1458:
        print(f"\n¡Solución óptima (1458) encontrada en la generación {i}!")
        break

print(f"\nProceso terminado en la generación {i+1}.")

Generaciones: 
99 / 100
Proceso terminado en la generación 100.


In [ ]:
**Resultados

In [7]:
import numpy as np

print(f"La mejor solución encontrada es: {MejorSolucion}")

# Calcular el peso final de la mejor solucion
peso_final = sum(p * v for p, v in zip(pesos, MejorSolucionVector))

print(f"Peso de la solución: {peso_final} (Capacidad Máx: {CapacidadMochila})")
print(f"Vector de solución (1 = objeto seleccionado):")
print(MejorSolucionVector)

items_seleccionados = [i for i, v in enumerate(MejorSolucionVector) if v == 1]
print(f"Índices de objetos seleccionados: {items_seleccionados}")

# Calcular y mostrar estadísticas adicionales
if all_gains: # Asegurarse de que la lista no este vacia
    print("\nEstadísticas de las soluciones encontradas:")
    print(f"Peor solución (ganancia mínima): {min(all_gains)}")
    print(f"Solución promedio (ganancia media): {np.mean(all_gains):.2f}")
    print(f"Desviación estándar de las ganancias: {np.std(all_gains):.2f}")
else:
    print("\nNo se generaron soluciones para calcular estadísticas.")

La mejor solución encontrada es: 844
Peso de la solución: 435 (Capacidad Máx: 750)
Vector de solución (1 = objeto seleccionado):
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1]
Índices de objetos seleccionados: [6, 10, 12, 14]

Estadísticas de las soluciones encontradas:
Peor solución (ganancia mínima): 0
Solución promedio (ganancia media): 210.80
Desviación estándar de las ganancias: 183.00


In [ ]:
# **Correciones Realizadas**

**Los problemas principales eran:**

  1. Manejo de listas: Estabas usando .append() en listas que ya tenían un tamaño fijo, lo que creaba listas cada vez más grandes y mezclaba ceros con los pesos/ganancias reales.

  2. Actualización de la solución: Actualizabas MejorSolucion dentro del bucle más interno, comparando soluciones a medio construir.

  3. Reparación: El intento de reparación (PesoHormigas[k] = 0) no funcionaba porque el objeto que querías quitar no estaba en el índice k.

  4. Actualización de feromonas: Reforzabas todos los objetos en cada generación, en lugar de reforzar selectivamente solo los que estaban en las mejores soluciones.